In [1]:
import numpy as np
import time
import tensorflow as tf

In [2]:
# iou 계산 함수
def iou(box1, box2): #box1, box2 -> center x, y / width , height
    """Compute the Intersection-Over-Union of two given boxes.

    Args:
    box1: array of 4 elements [cx, cy, width, height].
    box2: same as above
    Returns:
    iou: a float number in range [0, 1]. iou of the two boxes.
    """
    # left-right? box1,2 중 작은 오른쪽 x좌표 - 큰 왼쪽 x좌표 -> 겹치는 x구간
    lr = min(box1[0]+0.5*box1[2], box2[0]+0.5*box2[2]) - \
        max(box1[0]-0.5*box1[2], box2[0]-0.5*box2[2])
    if lr > 0:
        # top-bottom? box1,2 중 작은 아래쪽 y좌표 - 큰 위쪽 y좌표 -> 겹치는 구간
        tb = min(box1[1]+0.5*box1[3], box2[1]+0.5*box2[3]) - \
        max(box1[1]-0.5*box1[3], box2[1]-0.5*box2[3])
        if tb > 0: # 둘다 0보다 크다면 겹치는 구간이 존재함. 
            intersection = tb*lr #겹치는 구간 면적 계산
            union = box1[2]*box1[3]+box2[2]*box2[3]-intersection # 전체 면적 계산

            return intersection/union #결과 리턴 
    return 0

In [4]:
#배치로 IOU 계산 > box2는 아마 GT가 들어올 것 같음. box1은 여러 anchor? 
def batch_iou(boxes, box):
    """Compute the Intersection-Over-Union of a batch of boxes with another
    box.

    Args:
    box1: 2D array of [cx, cy, width, height].
    box2: a single array of [cx, cy, width, height]
    Returns:
    ious: array of a float number in range [0, 1].
    """
    lr = np.maximum(#가장 lr이 큰 것 (0보다 큰 것 중)
        np.minimum(boxes[:,0]+0.5*boxes[:,2], box[0]+0.5*box[2]) - \
        np.maximum(boxes[:,0]-0.5*boxes[:,2], box[0]-0.5*box[2]),
        0
    )
    tb = np.maximum( #가장 tb가 큰 것 (0보다 큰 것 중)
        np.minimum(boxes[:,1]+0.5*boxes[:,3], box[1]+0.5*box[3]) - \
        np.maximum(boxes[:,1]-0.5*boxes[:,3], box[1]-0.5*box[3]),
        0
    )
    inter = lr*tb #동일
    union = boxes[:,2]*boxes[:,3] + box[2]*box[3] - inter 
    return inter/union

In [5]:
# non-maximun supression.. 
def nms(boxes, probs, threshold):
    """Non-Maximum supression.
    Args:
    boxes: array of [cx, cy, w, h] (center format)
    probs: array of probabilities
    threshold: two boxes are considered overlapping if their IOU is largher than
        this threshold
    form: 'center' or 'diagonal'
    Returns:
    keep: array of True or False.
    """
    # probs는 각 박스에 대한 확률인데... confidence score? 인가 
    # arg sort는 sort를 하는데 arg로 나타냄. [2,1,3] > [1,0,2] 가 됨 
    # a.argsort()여기에 [::-1]이 붙으면 reverse가 되서 [2,0,1]이 됨. -1이 리버슨가..?
    order = probs.argsort()[::-1]
    #일단 초기화는 다 True로. 
    keep = [True]*len(order) 

    #keep 할건지 계산해보고 아니라면 False로 해당 arg를 변환해줌 
    for i in range(len(order)-1):
        ovps = batch_iou(boxes[order[i+1:]], boxes[order[i]]) # iou 각각계산(overlaps)
        # order가 지금 확률이 큰 box부터 arg 순서로 되어있으니.. 
        # 큰거 순서로 큰걸 중앙으로 보고 그것보다 작은것 쭉 계산하는듯.. 
        for j, ov in enumerate(ovps): #threshold 보다 작으면 다 False / enumerate는 index, value 리턴
            if ov > threshold:
                keep[order[j+i+1]] = False
                
                
    #결국 유지해줄 boxes들을 리턴하는 것인데. 
    #confidence score가 높은 친구들부터 차례대로 보면서 overlap이 threshold 이상인 친구들만 남김 트루
    #배치로 계산
    
    return keep

In [6]:
# TODO(bichen): this is not equivalent with full NMS. Need to improve it.
#라고 하니 이건 대충... 
def recursive_nms(boxes, probs, threshold, form='center'):
    """Recursive Non-Maximum supression.
    Args:
    boxes: array of [cx, cy, w, h] (center format) or [xmin, ymin, xmax, ymax]
    probs: array of probabilities
    threshold: two boxes are considered overlapping if their IOU is largher than
        this threshold
    form: 'center' or 'diagonal'
    Returns:
    keep: array of True or False.
    """
    
    #coordinates form  > center인지 diagonal인지..? 
    assert form == 'center' or form == 'diagonal', \
        'bounding box format not accepted: {}.'.format(form)

    #여기서는 보통 센터를 사용하는듯.  [cx, cy, w, h]  가 들어온다면 diagonal로 변경해줌
    if form == 'center':
        # convert to diagonal format
        boxes = np.array([bbox_transform(b) for b in boxes])

    # box 면적 계산 
    areas = (boxes[:, 2]-boxes[:, 0])*(boxes[:, 3]-boxes[:, 1])
    # x 작은 것 부터? left 부터 
    hidx = boxes[:, 0].argsort()
    #초기화 
    keep = [True]*len(hidx)
    
    #h index? arg sort array (x) 
    def _nms(hidx):
        order = probs[hidx].argsort()[::-1] #nms와 동일 

        for idx in range(len(order)): #돌면서 
            if not keep[hidx[order[idx]]]: # keep 이 true일 때만 아래로 내려감 
                continue
            
            xx2 = boxes[hidx[order[idx]], 2] # x index인데 order순서대로 > 가장 높은 prob 부터..
            for jdx in range(idx+1, len(order)):
                if not keep[hidx[order[jdx]]]: #여기도 True만 봄 
                    continue
                xx1 = boxes[hidx[order[jdx]], 0] 
                if xx2 < xx1:#xx2가 xx1보다 작으면 스톱 
                    break
                w = xx2 - xx1 #아니면 w 구해서 yy1,yy2 구하고 
                yy1 = max(boxes[hidx[order[idx]], 1], boxes[hidx[order[jdx]], 1])
                yy2 = min(boxes[hidx[order[idx]], 3], boxes[hidx[order[jdx]], 3])
                if yy2 <= yy1: #여기서도 yy2가 더 커야 논스톱 
                    continue
                h = yy2-yy1
                inter = w*h #구한 intersection. 
                iou = inter/(areas[hidx[order[idx]]]+areas[hidx[order[jdx]]]-inter)
                if iou > threshold: #iou가 th보다 크면 False.. 보통 반대 아닌가..? 
                    keep[hidx[order[jdx]]] = False

    def _recur(hidx):
        if len(hidx) <= 20: #hidx가 20보다 작으면 그냥 _mns 하고 
            _nms(hidx)
        else: #그거보다 크면 divide & conquer? 
            mid = len(hidx)/2
            _recur(hidx[:mid])
            _recur(hidx[mid:])
            _nms([idx for idx in hidx if keep[idx]])

    _recur(hidx)#recur함수 돌림. 

    return keep

In [7]:
#sparse를 dense matrix로 바꿔준다. 원핫인코딩 느낌? 
def sparse_to_dense(sp_indices, output_shape, values, default_value=0):
    """Build a dense matrix from sparse representations.

    Args:
    sp_indices: A [0-2]-D array that contains the index to place values.
    shape: shape of the dense matrix.
    values: A {0,1}-D array where values corresponds to the index in each row of
    sp_indices.
    default_value: values to set for indices not specified in sp_indices.
    Return:
    A dense numpy N-D array with shape output_shape.
    """
    #sp_indices 인덱스 포함한 array (matrix)
    #output_shape 출력 쉐입 
    #values 벨류> sp_indices의 각 row index에 해당 (class)
    #default 해당값 이외 부분 채울 값 
    #입력과 출력의 length 확인 
    assert len(sp_indices) == len(values), \
        'Length of sp_indices is not equal to length of values'
    #기본 default value로 세팅해줌. 
    array = np.ones(output_shape) * default_value
    for idx, value in zip(sp_indices, values): #해당 인덱스에 value 넣어줌 
        array[tuple(idx)] = value
    return array

"""
sp = [[1,2],[3,4],[1,1],[2,3],[4,4]]
print(sp)
sparse_to_dense(sp_indices=sp , output_shape=(5,5), values=[1,2,3,4,5])
array([[ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  3.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  4.,  0.],
       [ 0.,  0.,  0.,  0.,  2.],
       [ 0.,  0.,  0.,  0.,  5.]])
그냥 이런거 인듯

"""

In [9]:
def bgr_to_rgb(ims): #image bgr 형식을 rgb로 변환해줌. 
    """Convert a list of images from BGR format to RGB format."""
    out = []
    for im in ims:
        out.append(im[:,:,::-1])
    return out


In [8]:
def bbox_transform(bbox): #centor form을 diagonal form으로 변경해줌. 
    """convert a bbox of form [cx, cy, w, h] to [xmin, ymin, xmax, ymax]. Works
    for numpy array or list of tensors.
    """
    with tf.variable_scope('bbox_transform') as scope:
        cx, cy, w, h = bbox
        out_box = [[]]*4
        out_box[0] = cx-w/2
        out_box[1] = cy-h/2
        out_box[2] = cx+w/2
        out_box[3] = cy+h/2

    return out_box

In [10]:
def bbox_transform_inv(bbox): #diagonal form을 center form으로 변경해줌. 
    """convert a bbox of form [xmin, ymin, xmax, ymax] to [cx, cy, w, h]. Works
    for numpy array or list of tensors.
    """
    with tf.variable_scope('bbox_transform_inv') as scope:
        xmin, ymin, xmax, ymax = bbox
        out_box = [[]]*4

        width       = xmax - xmin + 1.0
        height      = ymax - ymin + 1.0
        out_box[0]  = xmin + 0.5*width 
        out_box[1]  = ymin + 0.5*height
        out_box[2]  = width
        out_box[3]  = height

    return out_box

In [11]:
class Timer(object): #타이머 클래스. 
    def __init__(self):
        self.total_time   = 0.0
        self.calls        = 0
        self.start_time   = 0.0
        self.duration     = 0.0
        self.average_time = 0.0

    def tic(self): #start time 체크 
        self.start_time = time.time()

    def toc(self, average=True): #끝 타임 - duration, 토탈, 콜 해서 avg타임도 잼
        self.duration = time.time() - self.start_time
        self.total_time += self.duration
        self.calls += 1
        self.average_time = self.total_time/self.calls
        if average:
            return self.average_time
        else:
            return self.duration

In [ ]:
def safe_exp(w, thresh): # 안전 익스포넨셜..? 
    """Safe exponential function for tensors."""

    slope = np.exp(thresh) # e ^ thresh
    with tf.variable_scope('safe_exponential'):
        lin_bool = w > thresh # w가 thresh보다 더 크면 
        lin_region = tf.to_float(lin_bool) #float으로 변환..? 

        lin_out = slope*(w - thresh + 1.)
        exp_out = tf.exp(tf.where(lin_bool, tf.zeros_like(w), w))

        out = lin_region*lin_out + (1.-lin_region)*exp_out
    return out

"""
nn_skeleton.py 
 box_center_x = tf.identity(
            anchor_x + delta_x * anchor_w, name='bbox_cx')
        box_center_y = tf.identity(
            anchor_y + delta_y * anchor_h, name='bbox_cy')
        box_width = tf.identity(
            anchor_w * util.safe_exp(delta_w, mc.EXP_THRESH),
            name='bbox_width')
        box_height = tf.identity(
            anchor_h * util.safe_exp(delta_h, mc.EXP_THRESH),
            name='bbox_height')
여기서 사용함 (1번식 같음)

config.py 내부 
 # threshold for safe exponential operation
  cfg.EXP_THRESH=1.0
  
델타는 뭐 값이지 않을까 함.. 
 delta_x, delta_y, delta_w, delta_h = tf.unstack(
            self.pred_box_delta, axis=2)
            여기서 delta가 나옴. 
"""

In [3]:
import numpy as np

In [4]:
_image_idx = [1,2,3,4,5]
_idx = [_image_idx[i] for i in np.random.permutation(np.arange(len(_image_idx)))]

In [5]:
_idx

[1, 3, 4, 2, 5]

In [10]:
im = np.array([[[1,2,3],[2,2,2],[1,1,1]],[[1,2,3],[5,5,5],[6,6,6]]])

In [11]:
im

array([[[1, 2, 3],
        [2, 2, 2],
        [1, 1, 1]],

       [[1, 2, 3],
        [5, 5, 5],
        [6, 6, 6]]])

In [12]:
im.shape

(2, 3, 3)

In [14]:
[float(v) for v in im.shape]

[2.0, 3.0, 3.0]

In [17]:
gt_bbox = np.array([[1,1,10,10], [10,55,25,99], [150,250,1222,1111]])
min(gt_bbox[:, 0] - gt_bbox[:, 2]/2.0+1)
gt_bbox[:, 0] - gt_bbox[:, 2]/2.0+1

array([  -3. ,   -1.5, -460. ])

In [21]:
np.random.randint(2)

0

In [23]:
a = np.array([1,2,3,4,5])
a[0::2]

array([1, 3, 5])

In [24]:
ad = set() 

In [25]:
ad

set()

In [27]:
ad.

{}